In [ ]:
import os
import pathlib  # for filepath path tooling
import lzma  # to decompress the iCOM file
import time
import asyncio

import numpy as np  # for array tooling
import pandas as pd
import matplotlib.pyplot as plt  # for plotting

import traitlets
import IPython
import ipywidgets

In [ ]:
# Makes it so that any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys

In [ ]:
icom_directory = pathlib.Path(r'\\physics-server\iComLogFiles\patients')
monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')

In [ ]:
output_directory = pathlib.Path(r'S:\Physics\Patient Specific Logfile Fluence')
pdf_directory = pathlib.Path(r'P:\Scanned Documents\RT\PhysChecks\Logfile PDFs')

In [ ]:
GRID = pymedphys.mudensity.grid()
COORDS = (GRID["jaw"], GRID["mlc"])

GAMMA_OPTIONS = {
    'dose_percent_threshold': 2,  # Not actually comparing dose though
    'distance_mm_threshold': 0.5,
    'local_gamma': True,
    'quiet': True,
    'max_gamma': 2,
}

In [ ]:
patient_id = '015231'

In [ ]:
def select_plans(patient_id):
    all_tel_paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/*tel.1'))
    all_tel_paths = sorted(all_tel_paths, key=os.path.getmtime)

    plan_names_to_choose_from = [
        f'{path.parent.name}/{path.name}' for path in all_tel_paths
    ]
    
    IPython.display.display(
        ipywidgets.SelectMultiple(
            options=plan_names_to_choose_from,
            description='Monaco',
            disabled=False
        )
    )

In [ ]:
select_plans(patient_id)

In [ ]:
def select_icom(patient_id):
    icom_deliveries = list(icom_directory.glob(f'{patient_id}_*/*.xz'))
    icom_deliveries = sorted(icom_deliveries, key=os.path.getmtime)
    
    icom_files_to_choose_from = [
        path.stem for path in icom_deliveries
    ]
    
    timestamps = pd.to_datetime(icom_files_to_choose_from, format='%Y%m%d_%H%M%S')
    
    IPython.display.display(
        ipywidgets.SelectMultiple(
            options=timestamps,
            description='Delivery',
            disabled=False
        )
    )

In [ ]:
list(pd.to_datetime(['20200202_111111'], format='%Y%m%d_%H%M%S').astype(str))

In [ ]:
select_icom(patient_id)

In [ ]:
def update_patient_id(patient_id):
    select_plans(patient_id)
    select_icom(patient_id)

In [ ]:
update_patient_id(patient_id)

In [ ]:
class Data(traitlets.HasTraits):
    patient_id = traitlets.Unicode()
    delivery_timestamp = traitlets.List(traitlets.Unicode())
    plan_names = traitlets.List(traitlets.Unicode())
    
data = Data()


def update_file_paths(change):
    patient_id = data.patient_id
    
    
    all_tel_paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/*tel.1'))
    all_tel_paths = sorted(all_tel_paths, key=os.path.getmtime)

    plan_names_to_choose_from = [
        f'{path.parent.name}/{path.name}' for path in all_tel_paths
    ]
    
    icom_deliveries = list(icom_directory.glob(f'{patient_id}_*/*.xz'))
    icom_deliveries = sorted(icom_deliveries, key=os.path.getmtime)
    
    icom_files_to_choose_from = [
        path.stem for path in icom_deliveries
    ]
    
    timestamps = list(pd.to_datetime(icom_files_to_choose_from, format='%Y%m%d_%H%M%S').astype(str))
    
    data.delivery_timestamp = timestamps
    data.plan_names = plan_names_to_choose_from
    
    
data.observe(update_file_paths, names=['patient_id'])

In [ ]:
data.patient_id = '015231'

In [ ]:
data.plan_names

In [ ]:
data.delivery_timestamp

In [ ]:
monaco_select = ipywidgets.SelectMultiple(
    options=data.plan_names,
    description='Monaco',
    disabled=False
)

def handle_monaco_select_change(change):
    monaco_select.options = data.plan_names
    

data.observe(handle_monaco_select_change, names=['plan_names'])

In [ ]:
icom_select = ipywidgets.SelectMultiple(
    options=data.delivery_timestamp,
    description='Delivery',
    disabled=False
)

def handle_icom_select_change(change):
    icom_select.options = data.delivery_timestamp
    

data.observe(handle_icom_select_change, names=['delivery_timestamp'])

In [ ]:
data.patient_id = '014877'

In [ ]:
monaco_select

In [ ]:
icom_select

In [ ]:


class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
        return debounced
    return decorator

In [ ]:
monaco_select.value

In [ ]:
IPython.display.display(

)

In [ ]:
IPython.display.display(

)

In [ ]:
monaco_plan_and_file_name = 'nosereplan/tel.1'  # plan directory and file name
icom_deliveries = ['20200326_111939']  # iCOM timestamps

In [ ]:
tel_path = list(monaco_directory.glob(f'*~{patient_id}/plan/{monaco_plan_and_file_name}'))[-1]
tel_path

In [ ]:
icom_paths = []

for icom_delivery in icom_deliveries:
    icom_paths += list(icom_directory.glob(f'{patient_id}_*/{icom_delivery}.xz'))

icom_paths

In [ ]:
icom_streams = []

for icom_path in icom_paths:
    with lzma.open(icom_path, 'r') as f:
        icom_streams += [f.read()]

In [ ]:
# Print out available methods and attributes on the Delivery object
[command for command in dir(pymedphys.Delivery) if not command.startswith('_')]

In [ ]:
deliveries_icom = []

for icom_stream in icom_streams:
    deliveries_icom += [pymedphys.Delivery.from_icom(icom_stream)]

In [ ]:
delivery_tel = pymedphys.Delivery.from_monaco(tel_path)

In [ ]:
mudensity_tel = delivery_tel.mudensity()

In [ ]:
mudensity_icom = np.zeros_like(mudensity_tel)

for path, delivery_icom in zip(icom_paths, deliveries_icom):
    print(path)
    mudensity_icom = mudensity_icom + delivery_icom.mudensity()

In [ ]:
def to_tuple(array):
    return tuple(map(tuple, array))

gamma = pymedphys.gamma(
    COORDS,
    to_tuple(mudensity_tel),
    COORDS,
    to_tuple(mudensity_icom),
    **GAMMA_OPTIONS
)

In [ ]:
def plot_gamma_hist(gamma, percent, dist):
    valid_gamma = gamma[~np.isnan(gamma)]

    plt.hist(valid_gamma, 50, density=True)
    pass_ratio = np.sum(valid_gamma <= 1) / len(valid_gamma)

    plt.title(
        "Local Gamma ({0}%/{1}mm) | Percent Pass: {2:.2f} % | Max Gamma: {3:.2f}".format(
            percent, dist, pass_ratio * 100, np.max(valid_gamma)
        )
    )

In [ ]:
def plot_and_save_results(
    mudensity_tel,
    mudensity_icom,
    gamma,
    png_filepath,
    pdf_filepath,
    header_text="",
    footer_text="",
):
    diff = mudensity_icom - mudensity_tel
    largest_item = np.max(np.abs(diff))

    widths = [1, 1]
    heights = [0.3, 1, 1, 1, 0.1]
    gs_kw = dict(width_ratios=widths, height_ratios=heights)

    fig, axs = plt.subplots(5, 2, figsize=(10, 16), gridspec_kw=gs_kw)
    gs = axs[0, 0].get_gridspec()

    for ax in axs[0, 0:]:
        ax.remove()

    for ax in axs[1, 0:]:
        ax.remove()

    for ax in axs[4, 0:]:
        ax.remove()

    axheader = fig.add_subplot(gs[0, :])
    axhist = fig.add_subplot(gs[1, :])
    axfooter = fig.add_subplot(gs[4, :])

    axheader.axis("off")
    axfooter.axis("off")

    axheader.text(0, 0, header_text, ha="left", wrap=True, fontsize=30)
    axfooter.text(0, 1, footer_text, ha="left", va="top", wrap=True, fontsize=6)

    plt.sca(axs[2, 0])
    pymedphys.mudensity.display(GRID, mudensity_tel)
    axs[2, 0].set_title("Monaco Plan MU Density")

    plt.sca(axs[2, 1])
    pymedphys.mudensity.display(GRID, mudensity_icom)
    axs[2, 1].set_title("Recorded iCOM MU Density")

    plt.sca(axs[3, 0])
    pymedphys.mudensity.display(
        GRID, diff, cmap="seismic", vmin=-largest_item, vmax=largest_item
    )
    plt.title("iCOM - Monaco")

    plt.sca(axs[3, 1])
    pymedphys.mudensity.display(GRID, gamma, cmap="coolwarm", vmin=0, vmax=2)
    plt.title(
        "Local Gamma | "
        f"{GAMMA_OPTIONS['dose_percent_threshold']}%/"
        f"{GAMMA_OPTIONS['distance_mm_threshold']}mm")

    plt.sca(axhist)
    plot_gamma_hist(
        gamma, 
        GAMMA_OPTIONS['dose_percent_threshold'], 
        GAMMA_OPTIONS['distance_mm_threshold'])

    return fig

In [ ]:
results_dir = output_directory.joinpath(patient_id, tel_path.parent.name, icom_path.stem)
results_dir.mkdir(exist_ok=True, parents=True)

header_text = (
    f"Patient ID: {patient_id}\n"
    f"Plan Name: {tel_path.parent.name}\n"
)

icom_path_strings = '\n    '.join([str(icom_path) for icom_path in icom_paths])

footer_text = (
    f"tel.1 file path: {str(tel_path)}\n"
    f"icom file path(s): {icom_path_strings}\n"
    f"results path: {str(results_dir)}"
)

png_filepath = str(results_dir.joinpath("result.png").resolve())
pdf_filepath = str(pdf_directory.joinpath(f"{patient_id}-{monaco_plan_and_file_name.replace('/','-')}.pdf").resolve())

fig = plot_and_save_results(
    mudensity_tel, mudensity_icom, 
    gamma, png_filepath, pdf_filepath, 
    header_text=header_text, footer_text=footer_text
)

fig.tight_layout()
plt.savefig(png_filepath, dpi=300)
plt.show()

In [ ]:
!magick convert "{png_filepath}" "{pdf_filepath}"